In [2]:
import os
import fnmatch
import tarfile
import shutil

In [3]:
FREESURFER_HOME = '/usr/local/freesurfer/7.4.1-1'
%env FREESURFER_HOME = {FREESURFER_HOME}
%env SUBJECTS_DIR={FREESURFER_HOME}/subjects
%env FUNCTIONALS_DIR={FREESURFER_HOME}/sessions

env: FREESURFER_HOME=/usr/local/freesurfer/7.4.1-1
env: SUBJECTS_DIR=/usr/local/freesurfer/7.4.1-1/subjects
env: FUNCTIONALS_DIR=/usr/local/freesurfer/7.4.1-1/sessions


In [4]:
%env FS_OVERRIDE=0
FSFAST_HOME = FREESURFER_HOME + '/fsfast'
%env FSFAST_HOME={FSFAST_HOME}
%env MINC_BIN_DIR={FREESURFER_HOME}/mni/bin
%env MNI_DIR={FREESURFER_HOME}/mni
%env MINC_LIB_DIR={FREESURFER_HOME}/mni/lib
%env MNI_DATAPATH={FREESURFER_HOME}/mni/data
%env FREESURFER_HOME={FREESURFER_HOME}
%env LOCAL_DIR={FREESURFER_HOME}/local
%env FSF_OUTPUT_FORMAT=nii.gz
%env FMRI_ANALYSIS_DIR=$FSFAST_HOME
MNI_PERL5LIB = FREESURFER_HOME + "/mni/share/perl5"
%env MNI_PERL5LIB={MNI_PERL5LIB}
%env PERL5LIB={MNI_PERL5LIB}
%env FREESURFER={FREESURFER_HOME}
%env FIX_VERTEX_AREA=

env: FS_OVERRIDE=0
env: FSFAST_HOME=/usr/local/freesurfer/7.4.1-1/fsfast
env: MINC_BIN_DIR=/usr/local/freesurfer/7.4.1-1/mni/bin
env: MNI_DIR=/usr/local/freesurfer/7.4.1-1/mni
env: MINC_LIB_DIR=/usr/local/freesurfer/7.4.1-1/mni/lib
env: MNI_DATAPATH=/usr/local/freesurfer/7.4.1-1/mni/data
env: FREESURFER_HOME=/usr/local/freesurfer/7.4.1-1
env: LOCAL_DIR=/usr/local/freesurfer/7.4.1-1/local
env: FSF_OUTPUT_FORMAT=nii.gz
env: FMRI_ANALYSIS_DIR=/usr/local/freesurfer/7.4.1-1/fsfast
env: MNI_PERL5LIB=/usr/local/freesurfer/7.4.1-1/mni/share/perl5
env: PERL5LIB=/usr/local/freesurfer/7.4.1-1/mni/share/perl5
env: FREESURFER=/usr/local/freesurfer/7.4.1-1
env: FIX_VERTEX_AREA=


In [5]:
PATH = %env PATH
newPATH = PATH + ':' + FREESURFER_HOME + '/bin'+ ':' + FSFAST_HOME + '/bin' + ':' + FREESURFER_HOME + '/tktools'
%env PATH={newPATH}

env: PATH=/home/marquez/.conda/envs/MRtrix3/bin/.:/usr/lib64/ccache:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/usr/local/freesurfer/7.4.1-1/bin:/usr/local/freesurfer/7.4.1-1/fsfast/bin:/usr/local/freesurfer/7.4.1-1/tktools


In [6]:
Base_Dir = os.path.join((os.path.expanduser("~")), 'Documents/Visual-Studio/Image-Processing/')
DICOM_Dir = os.path.join(Base_Dir, "Images PUJ")
Output_Dir = os.path.join(Base_Dir, 'Outputs')
tmp_dir = os.path.join(Base_Dir, 'tmp')
os.makedirs(tmp_dir, 0o755)

In [7]:
Current_DIR_NAME = 'CONDFT'
Current_TAR_NAME = 'dicom_anonymized.tar.gz'
Current_DIR_PATH = os.path.join(DICOM_Dir, Current_DIR_NAME)

In [8]:
for folder in os.listdir(Current_DIR_PATH):
    file = os.path.join(Current_DIR_PATH, folder, Current_TAR_NAME)
    if os.path.isfile(file):
        file_output = os.path.join(Output_Dir, folder)
        file_tmp = os.path.join(tmp_dir, folder)
        os.makedirs(file_tmp, 0o755)
        os.makedirs(file_output, 0o755)
        print('Extracting ' + folder + '...')
        with tarfile.open(file, 'r:gz') as tar:
            tar.extractall(path=file_tmp)
        print('Converting ' + folder + '...')
        !dcm2niix n -o '{file_output}' '{file_tmp}' > /dev/null
        usefulFiles = fnmatch.filter(os.listdir(file_output), '*DTI*[!ADC].*[!json]') + fnmatch.filter(os.listdir(file_output), '*T1W3D*.nii')
        for item in os.listdir(file_output):
            if item not in usefulFiles:
                os.remove(os.path.join(file_output, item))
        shutil.rmtree(file_tmp)     
        print(folder + ' processed')

Extracting CONDFT006...
Converting CONDFT006...
CONDFT006 processed
Extracting CONDFT018...
Converting CONDFT018...
Error: Converted 7350 of 7556 files
CONDFT018 processed
Extracting CONDFT011...
Converting CONDFT011...
CONDFT011 processed
Extracting CONDFT004...
Converting CONDFT004...
CONDFT004 processed
Extracting CONDFT003...
Converting CONDFT003...
CONDFT003 processed
Extracting CONDFT012...
Converting CONDFT012...
CONDFT012 processed
Extracting CONDFT090...
Converting CONDFT090...
CONDFT090 processed
Extracting CONDFT033...
Converting CONDFT033...
Error: Check sorted order: 4D dataset has 34 volumes, but volume index ranges from 33..3960
Error: Converted 17890 of 18406 files
CONDFT033 processed
Extracting CONDFT001...
Converting CONDFT001...
Error: Converted 18160 of 18367 files
CONDFT001 processed
Extracting CONDFT023...
Converting CONDFT023...
Error: Check sorted order: 4D dataset has 34 volumes, but volume index ranges from 33..3960
Error: Converted 18200 of 18406 files
CONDFT

In [9]:
for folder in os.listdir(Output_Dir):
    T1 = fnmatch.filter(os.listdir(os.path.join(Output_Dir, folder)), '*T1W3D*.nii')
    sub = 'sub-' + folder
    if len(T1) == 1:
        T1_PATH = os.path.join(Output_Dir, folder, T1[0])
        if os.path.isfile(T1_PATH):
            print('Freesurfer processing ' + folder + '...')
            !recon-all -i '{T1_PATH}' -s '{sub}' -all > /dev/null

Freesurfer processing DFT00076...
egrep: warning: egrep is obsolescent; using grep -E
egrep: warning: egrep is obsolescent; using grep -E
Freesurfer processing DFT00024...
egrep: warning: egrep is obsolescent; using grep -E
egrep: warning: egrep is obsolescent; using grep -E
Freesurfer processing DFT00072_2...
egrep: warning: egrep is obsolescent; using grep -E
egrep: warning: egrep is obsolescent; using grep -E
Freesurfer processing DFT00030...
egrep: warning: egrep is obsolescent; using grep -E
egrep: warning: egrep is obsolescent; using grep -E
Freesurfer processing DFT00060...
egrep: warning: egrep is obsolescent; using grep -E
egrep: warning: egrep is obsolescent; using grep -E
Freesurfer processing DFT00049...
egrep: warning: egrep is obsolescent; using grep -E
egrep: warning: egrep is obsolescent; using grep -E
Freesurfer processing CONDFT018...
egrep: warning: egrep is obsolescent; using grep -E
egrep: warning: egrep is obsolescent; using grep -E
egrep: warning: egrep is obsole